In [ ]:
# Colab setup (uncomment if running fresh)
# !python -m pip install -U pip
#pip install spacy==3.* transformers==4.* tokenizers==0.* sentencepiece==0.* beautifulsoup4==4.* lxml==5.* regex==2024.* unidecode==1.*

# Download a small spaCy model (English)
!#python -m spacy download en_core_web_sm


In [ ]:
#Classical tokenization
import re

text = "I can't believe it's not butter! Visit https://example.org, or email info@example.com."

# Whitespace-based tokens (baseline)
ws_tokens = text.split()

# Basic rule: split punctuation as separate tokens, keep contractions intact
# rule_tokens = re.findall(r"[A-Za-z]+(?:'[A-Za-z]+)?|[0-9]+|[^\sA-Za-z0-9]", text)

print("Whitespace:", ws_tokens)
# print("Rule-based:", rule_tokens)


Whitespace: ['I', "can't", 'believe', "it's", 'not', 'butter!', 'Visit', 'https://example.org,', 'or', 'email', 'info@example.com.']


In [ ]:
# Compare tokenizers (spaCy, BERT, GPT-2, T5)
import spacy
from transformers import AutoTokenizer

text = "I can’t believe it’s not butter!"  # note: curly apostrophes

# spaCy
nlp = spacy.load("en_core_web_sm")
spacy_toks = [t.text for t in nlp(text)]

# BERT (WordPiece)
bert_tok = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_toks = bert_tok.convert_ids_to_tokens(bert_tok.encode(text, add_special_tokens=False))

# GPT-2 (byte-level BPE)
gpt2_tok = AutoTokenizer.from_pretrained("gpt2")
gpt2_toks = gpt2_tok.convert_ids_to_tokens(gpt2_tok.encode(text, add_special_tokens=False))

# T5 (SentencePiece / Unigram LM)
t5_tok = AutoTokenizer.from_pretrained("t5-small")
t5_toks = t5_tok.convert_ids_to_tokens(t5_tok.encode(text, add_special_tokens=False))

print("spaCy:", spacy_toks)
print("BERT:", bert_toks)
print("GPT-2:", gpt2_toks)
print("T5:", t5_toks)


In [ ]:
bert_toks2 = bert_tok.convert_ids_to_tokens(bert_tok.encode("I just bought my Apple watch!", add_special_tokens=False))
print(bert_toks2)


['i', 'just', 'bought', 'my', 'apple', 'watch', '!']


In [ ]:
# Unicode normalization (NFC vs NFKC)
import unicodedata

s1 = "café"                     # composed: 'é' U+00E9
s2 = "cafe\u0301"               # decomposed: 'e' + COMBINING ACUTE

def show_norm(s):
    return {
        "orig": s,
        "NFC": unicodedata.normalize("NFC", s),
        "NFD": unicodedata.normalize("NFD", s),
        "NFKC": unicodedata.normalize("NFKC", s),
        "NFKD": unicodedata.normalize("NFKD", s),
    }

print(show_norm(s1))
print(show_norm(s2))

print("Equality (raw):", s1 == s2)
print("Equality (NFC):", unicodedata.normalize("NFC", s1) == unicodedata.normalize("NFC", s2))


{'orig': 'café', 'NFC': 'café', 'NFD': 'café', 'NFKC': 'café', 'NFKD': 'café'}
{'orig': 'café', 'NFC': 'café', 'NFD': 'café', 'NFKC': 'café', 'NFKD': 'café'}
Equality (raw): False
Equality (NFC): True


In [ ]:
# Regex cleaning (remove emails/URLs/HTML)
import re
from bs4 import BeautifulSoup

raw = """
<p>Contact us at <b>info@example.com</b>! Visit https://example.org or <a href="https://example.com">our site</a>.
<div>New iPhones!!! <em>Limited</em> stock.</div>
"""

# Strip HTML tags (BeautifulSoup) → text only
text = BeautifulSoup(raw, "lxml").get_text(separator=" ")

# # Remove emails and URLs
text = re.sub(r"\S+@\S+\.\S+", "", text)            # emails
text = re.sub(r"http[s]?://\S+", "", text)          # urls

# # Collapse extra whitespace
# text = re.sub(r"\s+", " ", text).strip()

print(text)


Contact us at   ! Visit  or  our site .
 New iPhones!!!  Limited  stock. 



In [ ]:
# Regex Extraction
import re

doc = "Conference dates: 2024-10-12 and 2025-01-05. Ticket IDs: A-12345, B-987."
dates = re.findall(r"\b\d{4}-\d{2}-\d{2}\b", doc)
ticket_ids = re.findall(r"\b[A-Z]-\d{3,6}\b", doc)

print("Dates:", dates)
print("Ticket IDs:", ticket_ids)


Dates: ['2024-10-12', '2025-01-05']
Ticket IDs: ['A-12345', 'B-987']


In [ ]:
# Stopwords & punctuation choices (spaCy)
import spacy
nlp = spacy.load("en_core_web_sm")

sent = "This is really not great!!! Honestly, I’m shocked."

doc = nlp(sent)
keep_all = [t.text for t in doc]
no_stop = [t.text for t in doc if not t.is_stop]
no_punct = [t.text for t in doc if not t.is_punct]
lemma_lower_no_stop_punct = [t.lemma_.lower() for t in doc if not t.is_stop and not t.is_punct]

print("Original:", keep_all)
print("No stopwords:", no_stop)
print("No punctuation:", no_punct)
print("Lemma+lower, no stop/punct:", lemma_lower_no_stop_punct)


Original: ['This', 'is', 'really', 'not', 'great', '!', '!', '!', 'Honestly', ',', 'I', '’m', 'shocked', '.']
No stopwords: ['great', '!', '!', '!', 'Honestly', ',', 'shocked', '.']
No punctuation: ['This', 'is', 'really', 'not', 'great', 'Honestly', 'I', '’m', 'shocked']
Lemma+lower, no stop/punct: ['great', 'honestly', 'shocked']


In [ ]:
# End-to-end preprocessing pipeline (spaCy + regex)
import re, unicodedata, spacy

nlp = spacy.load("en_core_web_sm")

URL_RE = re.compile(r"http\S+")
HTML_RE = re.compile(r"<.*?>", flags=re.S)

def preprocess(text: str):
    # 1) Unicode normalization (NFKC)
    text = unicodedata.normalize("NFKC", text)

    # 2) Remove URLs & HTML
    text = URL_RE.sub("", text)
    text = HTML_RE.sub("", text)

    # 3) spaCy processing
    doc = nlp(text)

    # 4) Normalize: case-fold + lemmatize, filter stop/punct/space
    out = []
    for tok in doc:
        if tok.is_stop or tok.is_punct or tok.is_space:
            continue
        lemma = tok.lemma_.lower().strip()
        # if lemma: // may be necessary for large document
        out.append(lemma)
    return out

sample = "Apple is releasing the new iPhone!!! Visit https://apple.com for details."
print(preprocess(sample))
# Expected-style output: ['apple', 'release', 'new', 'iphone', 'visit', 'detail']


['apple', 'release', 'new', 'iphone', 'visit', 'detail']


In [ ]:
# Tiny helper to show each pipeline stage
def show_stages(text: str):
    import re, unicodedata, spacy
    nlp = spacy.load("en_core_web_sm")
    URL_RE = re.compile(r"http\S+")
    HTML_RE = re.compile(r"<.*?>", flags=re.S)

    print("RAW:", text)
    t1 = unicodedata.normalize("NFKC", text)
    print("NFKC:", t1)
    t2 = URL_RE.sub("", HTML_RE.sub("", t1))
    print("CLEAN:", t2)
    doc = nlp(t2)
    print("TOKENS:", [t.text for t in doc])
    lemmas = [t.lemma_.lower() for t in doc if not (t.is_stop or t.is_punct or t.is_space)]
    print("LEMMA+FILTERED:", lemmas)

show_stages("Apple is releasing the new iPhone!!! Visit https://apple.com for details.")


RAW: Apple is releasing the new iPhone!!! Visit https://apple.com for details.
NFKC: Apple is releasing the new iPhone!!! Visit https://apple.com for details.
CLEAN: Apple is releasing the new iPhone!!! Visit  for details.
TOKENS: ['Apple', 'is', 'releasing', 'the', 'new', 'iPhone', '!', '!', '!', 'Visit', ' ', 'for', 'details', '.']
LEMMA+FILTERED: ['apple', 'release', 'new', 'iphone', 'visit', 'detail']
